In [1]:
# # code required to run on a fresh install or in google colab
# ! git clone https://github.com/CPMpy/XCP-explain.git /tmp/XCP-explain
# ! cd XCP-explain && git checkout ecai24
# ! pip install -r XCP-explain/requirements.txt
# ! pip install cpmpy

# # add XCP-explain to the Python path
# import sys
# root = "/tmp/XCP-explain"
# if root not in sys.path:
#     sys.path.insert(0, root)
root= "."

In [2]:
%load_ext autoreload
%autoreload 2
"""
    Some imports used throughout the notebook
"""
import time
import os
from visualize import *

from cpmpy.transformations.normalize import toplevel_list
from factory import *
from read_data import get_data
from IPython.display import clear_output


import numpy as np
np.set_printoptions(linewidth=90)
# preload solvers
from cpmpy import SolverLookup
names = SolverLookup.solvernames()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-03


## Hands-on causal explanations

In this notebook, you will get experience with computing causal explanations.

We focus on explaining unsatisfiability by means of extracting a MUS.



<img src="img/mus.png" width="20%" align="right" style="margin:50px;">

Trim model to minimal set of constraints

... minimize cognitive burden for user

In [3]:
instance = os.path.join(root,"Benchmarks/Instance1.txt")
data = get_data(instance)
factory = NurseSchedulingFactory(data)

Let's have a look at the instance and its (optimal) solution.

In [4]:
data.staff[["name", "MaxShifts","MaxWeekends"]]

,name,MaxShifts,MaxWeekends
0,Megan,D=14,1
1,Katherine,D=14,1
2,Robert,D=14,1
3,Jonathan,D=14,1
4,William,D=14,1
5,Richard,D=14,1
6,Kristen,D=14,1
7,Kevin,D=14,1


In [5]:
print(f"Planning for {data.horizon} days")

Planning for 14 days


In [6]:
model, nurse_view = factory.get_optimization_model()
assert model.solve(solver="ortools") # you can try different solvers here!


print(model.status())
print("Total penalty:", model.objective_value())
visualize(nurse_view.value(), factory)

ExitStatus.OPTIMAL (0.087745 seconds)
Total penalty: 607


In [7]:
requests, _ = factory.shift_on_requests(formulation="hard")

denied_requests = [req for req in requests if req.value() is False]
print("The following requests were denied:")
for req in denied_requests:
    print("-", req)

visualize_constraints(denied_requests, nurse_view, factory, do_clear=False)

The following requests were denied:
- Robert requests to work shift D on Thu 1
- Robert requests to work shift D on Fri 1
- Kevin requests to work shift D on Sat 2
- Kevin requests to work shift D on Sun 2


In [8]:
# try it yourself!

# requests, _ = factory.shift_off_requests(formulation="hard")
# cover_constraints, _ = factory.cover(formulation="hard")

# TODO: find out which are not satisfied, and visualize!

## Deductive explanations

In the remainder of this notebook, we will explore different ways of explaining unsatisfiabily of the instance.

In [9]:
model, nurse_view = factory.get_decision_model()
model.solve()

False

In [10]:
from cpmpy.tools.explain import mus

t0 = time.time()
conflict = mus(model.constraints) # try different solvers here!
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

Found conflict of size 11 in 0.69s


In [11]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

- Robert requests to work shift D on Wed 1
- Shift D on Sat 1 must be covered by 5 nurses out of 8
- Robert can work at most 5 days before having a day off
- Kevin should work at most 1 weekends
- Robert requests to work shift D on Tue 1
- Robert requests to work shift D on Thu 1
- Kevin requests to work shift D on Sat 2
- Robert requests to work shift D on Fri 1
- Katherine has a day off on Sat 1
- Robert requests to work shift D on Mon 1
- Richard has a day off on Sat 1


Now, let's influence the MUS we would like to find.

We can chose from QuickXplain [1] or Optimal MUS (OUS) [2]

**QuickXplain** takes as input a total ordering of constraints, and returns a lexicographically minimal MUS.
The algorithm is build up as a divide-and-conquer approach, and therefore has a good average complexity.

**OUS** takes as input a weight for each constraint, and finds a **optimal** MUS. While this optimality guarantee is sometimes required, it comes at a penalty of longer computation times, as you will notice here!

In [12]:
# QuickXplain first
from cpmpy.tools.explain import quickxplain


def get_weight(cons):
    if "William" in str(cons): # Find a different MUS than the previous
        return 2 
    return 1

ordered = sorted(model.constraints, key=get_weight)
conflict = quickxplain(ordered)
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

Found conflict of size 11 in 3.1s


In [13]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

- Robert can work at most 5 days before having a day off
- Kevin should work at most 1 weekends
- Katherine has a day off on Sat 1
- Richard has a day off on Sat 1
- Robert should have at least 2 consecutive days off
- Robert should have at least 2 consecutive days off
- Robert requests to work shift D on Mon 1
- Robert requests to work shift D on Tue 1
- Robert requests to work shift D on Thu 1
- Kevin requests to work shift D on Sat 2
- Shift D on Sat 1 must be covered by 5 nurses out of 8


In [14]:
# Now find truely OPTIMAL MUSes
## Careful, this takes a while if you are not using Exact!
from cpmpy.tools.explain import optimal_mus

def get_weight(cons):
    if "william" in str(cons).lower():
        return 5
    else:
        return 1
#     return len(str(cons)) # favor constraints with short description


solver = "exact" if "exact" in cp.SolverLookup.solvernames() else "ortools"
print("Using solver", solver)

conflict = optimal_mus(model.constraints, 
                       weights=[get_weight(c) for c in model.constraints],
                       solver=solver,
                       hs_solver="gurobi")
print(f"Found conflict of size {len(conflict)} in {round(time.time()-t0,2)}s")

Using solver exact
Found conflict of size 3 in 4.76s


In [15]:
for c in conflict:
    print("-", c)

visualize_constraints(conflict, nurse_view, factory)

- Robert has a day off on Tue 2
- Richard requests not to work shift D on Tue 2
- Shift D on Tue 2 must be covered by 7 nurses out of 8


## Part 2, fixing UNSAT models

Now that we know _why_ a model is UNSAT, we need to fix it.

In the presentation, several techniques are shown for doing so.

Below, you can find some skeleton code to play around with feasibiliy restoration techniques

In [16]:
model, nurse_view = factory.get_decision_model()
model.solve()

False

In [17]:
from cpmpy.tools.explain import mss_opt, mcs_opt

def get_weight(cons):
    if "cover" in str(cons):
        return 10
    return 1

# find Max-CSP solution
optimal_subset = mss_opt(model.constraints, hard=[],weights=[get_weight(c) for c in model.constraints])
mcs = set(model.constraints) - set(optimal_subset)
print("Found solution after dropping these constraints:")
for i,c in enumerate(mcs):
    print(f"{i}.", c)


Found solution after dropping these constraints:
0. Robert has a day off on Tue 2
1. Kevin should work at most 1 weekends
2. Robert should have at least 2 consecutive days off
3. Kristen has a day off on Tue 1
4. Richard requests not to work shift D on Tue 2
5. William should work at most 1 weekends
6. Kristen should work at most 1 weekends
7. Jonathan should work at most 1 weekends


In [18]:
assert cp.Model(optimal_subset).solve() is True
visualize(nurse_view.value(), factory)
visualize_constraints(mcs, nurse_view, factory, do_clear=False)

### Slack-based relaxation

Apart from dropping constraints, they can also be _relaxed_ when numeric

In [19]:
model, nurse_view, slack_under, slack_over = factory.get_slack_model()  # CMPpy Model

# TODO..